<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BLOCK 1 — Install and imports

!pip install -q pyscf gemmi cupy-cuda12x

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import gemmi
import cupy as cp
from pyscf import gto, dft

plt.rcParams["figure.dpi"] = 120

In [ ]:
# BLOCK 2 — Set up directories and copy CIFs into /content/ispH_structures

import shutil

BASE_DIR   = Path("/content")
STRUCT_DIR = BASE_DIR / "ispH_structures"
LAP_DIR    = BASE_DIR / "ispH_laplacian_npz"
RESULT_DIR = BASE_DIR / "ispH_results"

for d in (STRUCT_DIR, LAP_DIR, RESULT_DIR):
    d.mkdir(parents=True, exist_ok=True)

# These must exist in /content (upload via Colab file browser)
cif_names = ["3ZGL.cif", "3ZGN.cif"]

for fname in cif_names:
    src = BASE_DIR / fname
    if not src.exists():
        print(f"[WARN] {src} not found in /content — upload or fix name.")
        continue
    dst = STRUCT_DIR / fname
    print(f"Copying {src} → {dst}")
    shutil.copy(str(src), str(dst))

print("\nFiles in ispH_structures:")
for f in STRUCT_DIR.glob("*.cif"):
    print("  ", f.name)

Copying /content/3ZGL.cif → /content/ispH_structures/3ZGL.cif
Copying /content/3ZGN.cif → /content/ispH_structures/3ZGN.cif

Files in ispH_structures:
   3ZGL.cif
   3ZGN.cif


In [ ]:
# BLOCK 3 — Global constants & bin list (BINS_LIST)

L_BOX  = 3.0      # Å half-length of cube for density/Laplacian grid
N_GRID = 32       # 32^3 grid
MODE_B_SHIFT = 0.20  # Å
TRAP_THRESHOLD = 1e-9

# BINS_LIST exactly as in your STO-3G pipeline
BINS_LIST = [
    1,3,5,7,9,10,11,12,13,14,15,16,17,18,19,20,
    21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,
    36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
    51,52,53,54,55,56,57,58,59,60,
    65,70,71,73,75,77,78,79,80,81,82,83,84,85,
    87,88,89,90,91,92,93,94,95,96,97,98,99,100,
    102,103,105,106,107,109,110,111,112,113,114,
    115,116,117,118,119,120,121,122,123,124,125
]
L_EFF = 3.0       # for Δr = L_EFF / Nbins

print("Total bins:", len(BINS_LIST))

Total bins: 106


In [ ]:
# BLOCK 4 — CIF reading & Fe/S cluster extraction

def read_structure_any(path: Path):
    print(f"Reading {path}")
    return gemmi.read_structure(str(path))

def get_fe_s_clusters(struct, cutoff=3.0):
    """Return list of Fe/S connectivity clusters."""
    atoms = []
    for model in struct:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element.name.upper() in ("FE", "S"):
                        atoms.append(atom)
    if not atoms:
        raise RuntimeError("No Fe or S atoms found")

    coords = np.array([[a.pos.x, a.pos.y, a.pos.z] for a in atoms], float)
    n = len(atoms)
    adj = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if np.linalg.norm(coords[i] - coords[j]) <= cutoff:
                adj[i].append(j)
                adj[j].append(i)

    clusters = []
    visited = [False]*n
    for i in range(n):
        if visited[i]:
            continue
        stack=[i]; comp=[]
        visited[i]=True
        while stack:
            k=stack.pop()
            comp.append(atoms[k])
            for nb in adj[k]:
                if not visited[nb]:
                    visited[nb]=True
                    stack.append(nb)
        clusters.append(comp)

    print("  Found", len(clusters), "Fe/S clusters")
    return clusters

def select_fe4s4(clusters):
    """Select Fe4S4 cluster closest to overall Fe/S centroid."""
    cand=[]
    for comp in clusters:
        n_fe = sum(1 for a in comp if a.element.name.upper()=="FE")
        n_s  = sum(1 for a in comp if a.element.name.upper()=="S")
        if n_fe == 4 and n_s >= 4:
            cand.append(comp)
    if not cand:
        raise RuntimeError("No Fe4S4 cluster found")

    all_atoms = [a for c in cand for a in c]
    all_coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in all_atoms], float)
    global_cent = all_coords.mean(axis=0)

    best = None
    best_d = 1e9
    for comp in cand:
        coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in comp], float)
        c = coords.mean(axis=0)
        d = np.linalg.norm(c - global_cent)
        if d < best_d:
            best, best_d = comp, d

    print("  Selected Fe4S4 cluster with", len(best), "atoms")
    return best

def atoms_to_pyscf(cluster_atoms):
    coords = np.array([[a.pos.x,a.pos.y,a.pos.z] for a in cluster_atoms], float)
    centre = coords.mean(axis=0)
    coords = coords - centre
    pys_atoms = []
    for atom, (x,y,z) in zip(cluster_atoms, coords):
        pys_atoms.append((atom.element.name.capitalize(), (float(x),float(y),float(z))))
    return pys_atoms

In [ ]:
# BLOCK 5 — Mode-B asymmetric Fe–S stretch

def apply_mode_b(pys_atoms, shift=MODE_B_SHIFT):
    """
    Mode-B: move nearest S to two Fe atoms in opposite directions.
    """
    syms   = [a[0] for a in pys_atoms]
    coords = np.array([a[1] for a in pys_atoms], float)

    fe_idx = [i for i,s in enumerate(syms) if s.lower()=="fe"]
    s_idx  = [i for i,s in enumerate(syms) if s.lower()=="s"]

    if len(fe_idx) < 2 or len(s_idx) < 2:
        raise RuntimeError("Need ≥2 Fe and ≥2 S for Mode-B")

    new = coords.copy()
    sign = +1.0
    for fi in fe_idx[:2]:
        fe = coords[fi]
        best_j = None
        best_d = 1e9
        for sj in s_idx:
            d = np.linalg.norm(coords[sj] - fe)
            if d < best_d:
                best_j, best_d = sj, d
        vec = coords[best_j] - fe
        u   = vec / np.linalg.norm(vec)
        new[best_j] += sign * shift * u
        sign *= -1.0

    out = [(s,(float(x),float(y),float(z))) for s,(x,y,z) in zip(syms,new)]
    return out

In [ ]:
# BLOCK 6 — SCF: UKS/PBE/STO-3G (CPU)

def run_scf_sto3g(mol_atoms, spin=4):
    """
    UKS PBE / STO-3G for Fe4S4.
    spin=4 (S=2) for high-spin Fe4S4; stable convergence.
    """
    mol = gto.Mole()
    mol.build(
        atom=mol_atoms,
        basis="sto-3g",
        charge=0,
        spin=spin,
        verbose=3
    )
    mf = dft.UKS(mol)
    mf.xc = "PBE"
    mf.max_cycle = 80
    mf.diis_space = 12
    mf.level_shift = 0.3
    mf.init_guess = "minao"

    print(f"  Running UKS/STO-3G with S={spin/2}...")
    e = mf.kernel()
    if mf.converged:
        print("  SCF converged, E =", e)
        return mol, mf

    print("  SCF did not converge, trying Newton...")
    mf2 = mf.newton()
    e2 = mf2.kernel()
    if mf2.converged:
        print("  Newton converged, E =", e2)
        return mol, mf2

    raise RuntimeError("SCF did not converge (UKS + Newton).")

In [ ]:
# BLOCK 7 — GPU Laplacian & HOMO density

def compute_abs_laplacian_gpu(mol, mf, L=L_BOX, N=N_GRID):
    """
    GPU Laplacian:
    - CPU: AO & density
    - GPU: finite-difference Laplacian via CuPy
    """
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    print("  Evaluating AO and density on CPU...")
    ao = dft.numint.eval_ao(mol, coords)
    dm = mf.make_rdm1()
    rho = dft.numint.eval_rho(mol, ao, dm)
    rho_grid = rho.reshape(N, N, N)

    dx = xs[1]-xs[0]
    dy = ys[1]-ys[0]
    dz = zs[1]-zs[0]

    rho_cp = cp.asarray(rho_grid)

    print("  Computing Laplacian on GPU...")
    d2x = (cp.roll(rho_cp, -1, 0) - 2*rho_cp + cp.roll(rho_cp, 1, 0)) / (dx*dx)
    d2y = (cp.roll(rho_cp, -1, 1) - 2*rho_cp + cp.roll(rho_cp, 1, 1)) / (dy*dy)
    d2z = (cp.roll(rho_cp, -1, 2) - 2*rho_cp + cp.roll(rho_cp, 1, 2)) / (dz*dz)
    lap_cp = d2x + d2y + d2z

    kappa_cp = cp.abs(lap_cp).ravel()
    kappa_np = cp.asnumpy(kappa_cp).astype(np.float64)
    return kappa_np

def compute_homo_density(mol, mf, L=L_BOX, N=N_GRID):
    """
    Compute |ψ_HOMO(r)|^2 on the same grid (for HOMO 3D clouds).
    Uses α-spin HOMO of UKS.
    """
    xs = np.linspace(-L, L, N)
    ys = np.linspace(-L, L, N)
    zs = np.linspace(-L, L, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")
    coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

    ao = dft.numint.eval_ao(mol, coords)

    # UKS: mo_coeff[0] and mo_occ[0] are α-spin
    mo_coeff_a = mf.mo_coeff[0]
    mo_occ_a   = mf.mo_occ[0]
    occ_idx = np.where(mo_occ_a > 0.5)[0]
    homo_idx = occ_idx[-1]

    c = mo_coeff_a[:, homo_idx]
    psi = ao.dot(c)
    rho_homo = np.abs(psi)**2

    return rho_homo.astype(np.float64)  # flat N^3 array

In [ ]:
# BLOCK 8 — Run SCF + GPU Laplacian + HOMO; save comb NPZs

def save_comb_and_homo_npz(label, k_base, k_mode, h_base, h_mode, out_path: Path):
    np.savez_compressed(
        out_path,
        kappa_base=k_base,
        kappa_mode=k_mode,
        homo_base=h_base,
        homo_mode=h_mode,
        label=label,
    )
    print("  Saved comb+HOMO NPZ →", out_path)

STRUCTS = [
    ("3ZGL_TMBPP", "3ZGL.cif"),
    ("3ZGN_AMBPP", "3ZGN.cif"),
]

for label, fname in STRUCTS:
    print("\n=======================================")
    print("Structure:", label, "from", fname)
    cif_path = STRUCT_DIR / fname
    if not cif_path.exists():
        print("  [ERROR] CIF not found:", cif_path)
        continue

    st        = read_structure_any(cif_path)
    clusters  = get_fe_s_clusters(st, cutoff=3.0)
    core      = select_fe4s4(clusters)
    pys_base  = atoms_to_pyscf(core)

    # BASE
    print("  SCF: BASE geometry")
    mol_b, mf_b = run_scf_sto3g(pys_base, spin=4)
    print("  Laplacian: BASE (GPU)")
    k_base = compute_abs_laplacian_gpu(mol_b, mf_b)
    print("  HOMO: BASE")
    h_base = compute_homo_density(mol_b, mf_b)

    # MODE-B
    pys_dist = apply_mode_b(pys_base, shift=MODE_B_SHIFT)
    print("  SCF: MODE-B geometry")
    mol_d, mf_d = run_scf_sto3g(pys_dist, spin=4)
    print("  Laplacian: MODE-B (GPU)")
    k_mode = compute_abs_laplacian_gpu(mol_d, mf_d)
    print("  HOMO: MODE-B")
    h_mode = compute_homo_density(mol_d, mf_d)

    comb_npz = LAP_DIR / f"{label}_comb_sto3g.npz"
    save_comb_and_homo_npz(label, k_base, k_mode, h_base, h_mode, comb_npz)

print("\nDone. comb+HOMO NPZ files in:", LAP_DIR)
for f in LAP_DIR.glob("*.npz"):
    print("  ", f.name)


Structure: 3ZGL_TMBPP from 3ZGL.cif
Reading /content/ispH_structures/3ZGL.cif
  Found 16 Fe/S clusters
  Selected Fe4S4 cluster with 11 atoms
  SCF: BASE geometry
  Running UKS/STO-3G with S=2.0...
SCF not converged.
SCF energy = -7686.15123154545 after 80 cycles  <S^2> = 6.1450466  2S+1 = 5.0576859
  SCF did not converge, trying Newton...
converged SCF energy = -7754.7239504264  <S^2> = 14.147031  2S+1 = 7.588684
  Newton converged, E = -7754.723950426399
  Laplacian: BASE (GPU)
  Evaluating AO and density on CPU...
  Computing Laplacian on GPU...
  HOMO: BASE
  SCF: MODE-B geometry
  Running UKS/STO-3G with S=2.0...
SCF not converged.
SCF energy = -7701.46968279784 after 80 cycles  <S^2> = 6.0777741  2S+1 = 5.0310135
  SCF did not converge, trying Newton...

WARN: HOMO -0.08408219644198854 > LUMO -0.08582605982312311 was found in the canonicalized orbitals.

converged SCF energy = -7754.80880052223  <S^2> = 14.745637  2S+1 = 7.74484
  Newton converged, E = -7754.808800522229
  Lapla

In [ ]:
# BLOCK 9 — Compute Δteeth(Δr) from comb NPZs and plot overlay

def count_traps_from_hist(hist, threshold=TRAP_THRESHOLD):
    mask = hist < threshold
    n = len(mask)
    i = 0
    k = 0
    while i < n:
        if mask[i]:
            k += 1
            while i+1 < n and mask[i+1]:
                i += 1
        i += 1
    return k

def dteeth_from_comb_npz(npz_path):
    data = np.load(npz_path, allow_pickle=True)
    kb = data["kappa_base"]
    km = data["kappa_mode"]
    label = str(data["label"])

    kmin = min(float(kb.min()), float(km.min()))
    kmax = max(float(kb.max()), float(km.max()))

    dr_list = []
    dt_list = []

    print(f"\n=== Δteeth from {npz_path.name} ===")
    for b in BINS_LIST:
        edges = np.linspace(kmin, kmax, b+1)
        hb, _ = np.histogram(kb, bins=edges, density=True)
        hm, _ = np.histogram(km, bins=edges, density=True)

        n_b = count_traps_from_hist(hb, TRAP_THRESHOLD)
        n_m = count_traps_from_hist(hm, TRAP_THRESHOLD)
        dteeth = n_m - n_b
        dr = L_EFF / b

        dr_list.append(dr)
        dt_list.append(dteeth)
        print(f" bins={b:3d}  Δr={dr:7.4f} Å  base={n_b:2d}  mode={n_m:2d}  Δteeth={dteeth:+3d}")

    return label, np.array(dr_list), np.array(dt_list)

# Process both comb NPZs
results = {}
for label, fname in STRUCTS:
    comb_npz = LAP_DIR / f"{label}_comb_sto3g.npz"
    if comb_npz.exists():
        lab, dr, dt = dteeth_from_comb_npz(comb_npz)
        results[lab] = (dr, dt)
    else:
        print("[WARN] Missing comb NPZ for", label)

# Plot overlay of Δteeth(Δr)
plt.figure(figsize=(8,4))
for lab,(dr,dt) in results.items():
    plt.plot(dr, dt, "-o", ms=3, label=lab)

plt.axhline(0.0, color="k", lw=1)
plt.xscale("log")
plt.xlim(2e-2, 1e-1)
plt.xlabel("Δr (Å, log scale)")
plt.ylabel("Δteeth (distorted − base)")
plt.title("GQR-15 Δteeth(Δr) — 3ZGL (TMBPP) vs 3ZGN (AMBPP)")
plt.grid(True, which="both", alpha=0.25)
plt.legend(fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# BLOCK 10 — 3D HOMO clouds for base vs Mode-B

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

def plot_homo_cloud_3d(npz_path, title_prefix="", N=N_GRID, frac=0.03):
    """
    Plot 3D HOMO |ψ|^2 clouds for BASE and Mode-B from comb NPZ.
    frac = fraction of highest-density points to plot (e.g. 0.03 = top 3%).
    """
    data = np.load(npz_path, allow_pickle=True)
    if "homo_base" not in data or "homo_mode" not in data:
        raise ValueError(f"{npz_path} has no homo_base/homo_mode arrays")

    homo_base = data["homo_base"].reshape(N, N, N)
    homo_mode = data["homo_mode"].reshape(N, N, N)
    label = str(data["label"])

    xs = np.linspace(-L_BOX, L_BOX, N)
    ys = np.linspace(-L_BOX, L_BOX, N)
    zs = np.linspace(-L_BOX, L_BOX, N)
    X, Y, Z = np.meshgrid(xs, ys, zs, indexing="ij")

    Xf = X.ravel()
    Yf = Y.ravel()
    Zf = Z.ravel()
    hb = homo_base.ravel()
    hm = homo_mode.ravel()

    thr_b = np.quantile(hb, 1.0 - frac)
    thr_m = np.quantile(hm, 1.0 - frac)
    mask_b = hb >= thr_b
    mask_m = hm >= thr_m

    fig = plt.figure(figsize=(10,4))

    # BASE
    ax1 = fig.add_subplot(1,2,1, projection="3d")
    p1 = ax1.scatter(Xf[mask_b], Yf[mask_b], Zf[mask_b],
                     c=hb[mask_b], s=8, alpha=0.6, cmap="viridis")
    ax1.set_title(f"{title_prefix} HOMO |ψ|^2 — BASE")
    ax1.set_xlabel("x (Å)")
    ax1.set_ylabel("y (Å)")
    ax1.set_zlabel("z (Å)")
    fig.colorbar(p1, ax=ax1, shrink=0.7, label="|ψ|^2")

    # Mode-B
    ax2 = fig.add_subplot(1,2,2, projection="3d")
    p2 = ax2.scatter(Xf[mask_m], Yf[mask_m], Zf[mask_m],
                     c=hm[mask_m], s=8, alpha=0.6, cmap="viridis")
    ax2.set_title(f"{title_prefix} HOMO |ψ|^2 — Mode-B")
    ax2.set_xlabel("x (Å)")
    ax2.set_ylabel("y (Å)")
    ax2.set_zlabel("z (Å)")
    fig.colorbar(p2, ax=ax2, shrink=0.7, label="|ψ|^2")

    plt.tight_layout()
    plt.show()

# Now call for each structure:
for label, fname in STRUCTS:
    npz_path = LAP_DIR / f"{label}_comb_sto3g.npz"
    if npz_path.exists():
        plot_homo_cloud_3d(npz_path, title_prefix=label)
    else:
        print("[WARN] Missing comb NPZ for HOMO 3D plot:", npz_path)